In [ ]:
# First, let's install all the required packages
print("Installing required packages...")
!pip install -q transformers torch datasets accelerate peft rouge_score nltk scikit-learn numpy polars

Installing required packages...
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.9 MB/s eta 0:00:00


In [ ]:
# Import all necessary libraries
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd
import polars as pl
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from datasets import Dataset, DatasetDict
from peft import LoraConfig, get_peft_model, PeftModel
import json
import os
from typing import Dict, List, Tuple
import math
import warnings
warnings.filterwarnings('ignore')

print("All packages installed successfully!")

All packages installed successfully!


In [ ]:
# ------------2: CHECK GPU AND SETUP DEVICE----------

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("No GPU available, using CPU (training will be slower)")

Using device: cuda
GPU: NVIDIA A100-SXM4-40GB
Memory: 42.5 GB


In [ ]:
# ------------3: UNDERSTANDING PERPLEXITY----------

print("\n" + "="*60)
print("UNDERSTANDING PERPLEXITY")
print("="*60)

print("""
What is Perplexity?
Perplexity measures how well a language model predicts text.
Think of it as measuring how 'surprised' the model is by the text.

Formula: Perplexity = exp(average negative log-likelihood)
PPL = exp(-1/N * Σ log P(w_i | w_1, ..., w_{i-1}))

How to interpret:
• Lower perplexity = Better performance
• Higher perplexity = Model is more confused/surprised
• Perfect model would have perplexity = 1

Two types we'll use:
1. Standard Perplexity: Overall text prediction quality
2. Conditional Perplexity: How well model predicts answers given questions
""")

class PerplexityEvaluator:
    """
    Perplexity Evaluator for Language Models

    This class implements perplexity calculation for evaluating language models.
    Perplexity measures how well the model predicts the next token in a sequence.
    """

    def __init__(self, model, tokenizer, device):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        print("Perplexity Evaluator initialized!")

    def calculate_perplexity(self, text: str) -> float:
        """
        Calculate perplexity for a given text

        Steps:
        1. Tokenize the input text
        2. Get model predictions (logits)
        3. Calculate cross-entropy loss
        4. Convert loss to perplexity using exp()
        """
        # Tokenize the text
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
        input_ids = inputs["input_ids"].to(self.device)

        # Need at least 2 tokens for loss calculation
        if input_ids.size(1) < 2:
            return float('inf')

        with torch.no_grad():
            # Get model outputs with labels for loss calculation
            outputs = self.model(input_ids, labels=input_ids)

            # Extract loss (negative log-likelihood)
            loss = outputs.loss

            # Convert loss to perplexity
            perplexity = torch.exp(loss).item()

        return perplexity

    def calculate_conditional_perplexity(self, context: str, target: str) -> float:
        """
        Calculate conditional perplexity P(target | context)
        This measures how well the model predicts the target given the context

        Perfect for Q&A evaluation: P(answer | question)
        """
        # Combine context and target
        full_text = context + " " + target
        full_inputs = self.tokenizer(full_text, return_tensors="pt", truncation=True, max_length=512)
        context_inputs = self.tokenizer(context, return_tensors="pt", truncation=True, max_length=512)

        full_input_ids = full_inputs["input_ids"].to(self.device)
        context_length = context_inputs["input_ids"].size(1)

        if full_input_ids.size(1) <= context_length:
            return float('inf')

        with torch.no_grad():
            outputs = self.model(full_input_ids, labels=full_input_ids)
            logits = outputs.logits

            # Focus only on the target tokens (after context)
            target_logits = logits[0, context_length-1:-1, :]
            target_ids = full_input_ids[0, context_length:]

            # Calculate log probabilities
            log_probs = F.log_softmax(target_logits, dim=-1)
            target_log_probs = log_probs.gather(1, target_ids.unsqueeze(1)).squeeze(1)

            # Calculate average negative log-likelihood
            avg_nll = -target_log_probs.mean()

            # Convert to perplexity
            perplexity = torch.exp(avg_nll).item()

        return perplexity


UNDERSTANDING PERPLEXITY

What is Perplexity?
Perplexity measures how well a language model predicts text.
Think of it as measuring how 'surprised' the model is by the text.

Formula: Perplexity = exp(average negative log-likelihood)
PPL = exp(-1/N * Σ log P(w_i | w_1, ..., w_{i-1}))

How to interpret:
• Lower perplexity = Better performance
• Higher perplexity = Model is more confused/surprised
• Perfect model would have perplexity = 1

Two types we'll use:
1. Standard Perplexity: Overall text prediction quality
2. Conditional Perplexity: How well model predicts answers given questions



In [ ]:
# ------------4: LOAD THE MODEL AND TOKENIZER----------
from google.colab import userdata
hf_token  = userdata.get('secretName')
print("\n" + "="*60)
print("LOADING MODEL AND TOKENIZER")
print("="*60)

model_name = "HuggingFaceTB/SmolLM-1.7B-Instruct"
print(f"Loading model: {model_name}")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("Tokenizer loaded!")

# Add pad token if it doesn't exist
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("Added pad token")

# Load model with memory-efficient settings
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto" if device == "cuda" else None,
    use_auth_token=hf_token
)

print("Model loaded!")


LOADING MODEL AND TOKENIZER
Loading model: HuggingFaceTB/SmolLM-1.7B-Instruct


tokenizer_config.json:   0%|          | 0.00/3.59k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

...

Tokenizer loaded!


config.json:   0%|          | 0.00/738 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.42G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

Model loaded!


In [ ]:
# # ------------4: LOAD THE MODEL AND TOKENIZER----------

# print("\n" + "="*60)
# print("LOADING MODEL AND TOKENIZER")
# print("="*60)

# model_name = "meta-llama/Llama-3.2-1B-Instruct"
# print(f"Loading model: {model_name}")

# # Note: LLaMA models require authentication. Ensure you have a HuggingFace token set up.
# # You can set it in your environment or via huggingface_hub login
# from huggingface_hub import login

# # Load tokenizer
# tokenizer = AutoTokenizer.from_pretrained(
#     model_name,
#     use_auth_token=True
# )
# print("Tokenizer loaded!")

# # Add pad token if it doesn't exist
# if tokenizer.pad_token is None:
#     tokenizer.pad_token = tokenizer.eos_token
#     print("Added pad token")

# # Load model with memory-efficient settings
# base_model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.bfloat16,
#     device_map="auto" if device == "cuda" else None,
#     use_auth_token=hf_token
# )

# print("Model loaded!")


LOADING MODEL AND TOKENIZER
Loading model: meta-llama/Llama-3.2-1B-Instruct


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Tokenizer loaded!
Added pad token


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Model loaded!


In [ ]:
# Check memory footprint
if hasattr(model, 'get_memory_footprint'):
    memory_mb = model.get_memory_footprint() / 1e6
    print(f"Model memory footprint: {memory_mb:.2f} MB")

# Display model info
print(f"Model parameters: {model.num_parameters():,}")
print(f"Model architecture: {model.config.architectures}")

Model memory footprint: 3422.75 MB
Model parameters: 1,711,376,384
Model architecture: ['LlamaForCausalLM']


In [ ]:
# ------------5: LOAD AND EXPLORE THE ANIME DATASET----------

print("\n" + "="*60)
print("LOADING ANIME UNDERSTANDING DATASET")
print("="*60)

# Define all anime categories
anime_categories = [
    "chainsawman", "kurokonobasuke", "onepunch", "hellsing", "frieren", "aot",
    "naruto", "dr_stone", "gundam_00", "darling-in-the-franxx",
    "berserk", "evangelion", "onepiece"
]

# Load datasets using Polars
print("Loading datasets using Polars...")
dataset_dict = {}

for anime in anime_categories:
    try:
        splits = {
            'train': f'{anime}_dev.jsonl',
            'validation': f'{anime}_val.jsonl',
            'test': f'{anime}_test.jsonl'
        }

        # Load train split (you can extend to validation/test if needed)
        df = pl.read_ndjson(f'hf://datasets/theblackcat102/anime-understanding-dataset/{splits["train"]}')

        # Convert Polars DataFrame to HuggingFace Dataset
        dataset_dict[anime] = Dataset.from_pandas(df.to_pandas())

        print(f"Loaded {anime} dataset: {len(df)} questions")
    except Exception as e:
        print(f"Error loading {anime} dataset: {e}")
        continue


LOADING ANIME UNDERSTANDING DATASET
Loading datasets using Polars...
Loaded chainsawman dataset: 5 questions
Loaded kurokonobasuke dataset: 5 questions
Loaded onepunch dataset: 5 questions
Loaded hellsing dataset: 5 questions
Loaded frieren dataset: 5 questions
Loaded aot dataset: 5 questions
Loaded naruto dataset: 5 questions
Loaded dr_stone dataset: 5 questions
Loaded gundam_00 dataset: 5 questions
Loaded darling-in-the-franxx dataset: 5 questions
Loaded berserk dataset: 5 questions
Loaded evangelion dataset: 5 questions
Loaded onepiece dataset: 5 questions


In [ ]:
# Create a DatasetDict
dataset = DatasetDict(dataset_dict)

print("Dataset loaded!")
print(f"Dataset structure: {dataset}")

# Explore the dataset
for anime in dataset:
    anime_data = dataset[anime]
    print(f"{anime.capitalize()} questions: {len(anime_data)}")

    # Show a sample question
    sample = anime_data[0]
    print(f"\nSample Question from {anime}:")
    print(f"Question: {sample['question']}")
    print(f"A) {sample['A']}")
    print(f"B) {sample['B']}")
    print(f"C) {sample['C']}")
    print(f"D) {sample['D']}")
    print(f"Correct Answer: {sample['answer']}")
    print(f"Anime: {sample['anime']}")

Dataset loaded!
Dataset structure: DatasetDict({
    chainsawman: Dataset({
        features: ['question', 'A', 'B', 'C', 'D', 'answer', 'anime'],
        num_rows: 5
    })
    kurokonobasuke: Dataset({
        features: ['question', 'A', 'B', 'C', 'D', 'answer', 'anime'],
        num_rows: 5
    })
    onepunch: Dataset({
        features: ['question', 'A', 'B', 'C', 'D', 'answer', 'anime'],
        num_rows: 5
    })
    hellsing: Dataset({
        features: ['question', 'A', 'B', 'C', 'D', 'answer', 'anime'],
        num_rows: 5
    })
    frieren: Dataset({
        features: ['question', 'A', 'B', 'C', 'D', 'answer', 'anime'],
        num_rows: 5
    })
    aot: Dataset({
        features: ['question', 'A', 'B', 'C', 'D', 'answer', 'anime'],
        num_rows: 5
    })
    naruto: Dataset({
        features: ['question', 'A', 'B', 'C', 'D', 'answer', 'anime'],
        num_rows: 5
    })
    dr_stone: Dataset({
        features: ['question', 'A', 'B', 'C', 'D', 'answer', 'anime'],
 

In [ ]:
# ------------6: FORMAT DATASET FOR TRAINING----------

print("\n" + "="*60)
print("FORMATTING DATASET FOR TRAINING")
print("="*60)

def format_qa_example(example):
    """
    Convert multiple choice questions into conversational format
    This creates training examples that teach the model to answer anime questions
    """
    question = example['question']
    options = [
        f"A) {example['A']}",
        f"B) {example['B']}",
        f"C) {example['C']}",
        f"D) {example['D']}"
    ]
    correct_answer = example['answer']
    anime = example['anime']

    # Find the full correct answer text
    answer_map = {'A': example['A'], 'B': example['B'], 'C': example['C'], 'D': example['D']}
    correct_answer_text = answer_map[correct_answer]

    # Create training format using chat template
    messages = [
        {
            "role": "user",
            "content": f"Answer this question about {anime} anime:\n\n{question}\n\n" + "\n".join(options)
        },
        {
            "role": "assistant",
            "content": f"The correct answer is {correct_answer}) {correct_answer_text}"
        }
    ]

    return {"messages": messages, "anime": anime, "correct_answer": correct_answer}


FORMATTING DATASET FOR TRAINING


In [ ]:
def create_training_dataset(dataset, tokenizer):
    """Convert the dataset to training format"""
    print("Processing dataset...")

    formatted_examples = []

    # Process all anime splits
    for anime in dataset:
        for i, example in enumerate(dataset[anime]):
            formatted = format_qa_example(example)

            # Apply chat template
            text = tokenizer.apply_chat_template(
                formatted["messages"],
                tokenize=False,
                add_generation_prompt=False
            )

            formatted_examples.append({
                "text": text,
                "anime": formatted["anime"],
                "correct_answer": formatted["correct_answer"]
            })

            # Show progress
            if (i + 1) % 100 == 0:
                print(f"Processed {i + 1} examples from {anime}...")

    return Dataset.from_list(formatted_examples)

# Create the training dataset
train_dataset = create_training_dataset(dataset, tokenizer)
print(f"Training dataset created with {len(train_dataset)} examples!")

# Show a formatted example
print("\nSample Formatted Training Example:")
print(train_dataset[0]['text'][:500] + "...")

Processing dataset...
Training dataset created with 65 examples!

Sample Formatted Training Example:
<|im_start|>user
Answer this question about chainsawman anime:

Why does Denji refuse to kill Reze?

A) Because he still dreams of starting a new life with her
B) Because he is afraid of her powers
C) Because Makima ordered him not to
D) Because he believes she will turn good<|im_end|>
<|im_start|>assistant
The correct answer is A) Because he still dreams of starting a new life with her<|im_end|>
...


In [ ]:
# ------------7: TOKENIZE THE DATASET----------

print("\n" + "="*60)
print("TOKENIZING DATASET")
print("="*60)

def tokenize_function(examples):
    """Tokenize the text examples for training"""
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        padding=False
    )

print("Tokenizing dataset...")
tokenized_dataset = train_dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=train_dataset.column_names,
    desc="Tokenizing"
)

print(f"Dataset tokenized!")
print(f"Tokenized dataset size: {len(tokenized_dataset)}")


TOKENIZING DATASET
Tokenizing dataset...


Tokenizing:   0%|          | 0/65 [00:00<?, ? examples/s]

Dataset tokenized!
Tokenized dataset size: 65


In [ ]:
# Show tokenization stats
sample_tokens = tokenized_dataset[0]['input_ids']
print(f"Sample sequence length: {len(sample_tokens)} tokens")
print(f"Sample tokens: {sample_tokens[:20]}...")

Sample sequence length: 94 tokens
Sample tokens: [1, 4093, 198, 21350, 451, 1962, 563, 11966, 912, 1483, 39113, 42, 198, 198, 4898, 1072, 8732, 10381, 16907, 288]...


In [ ]:
# ------------8: SETUP LORA CONFIGURATION----------

print("\n" + "="*60)
print("SETTING UP LORA CONFIGURATION")
print("="*60)

# LoRA (Low-Rank Adaptation) allows efficient fine-tuning
# by adding small trainable matrices to existing layers
lora_config = LoraConfig(
    r=16,  # Rank - controls the bottleneck size
    lora_alpha=32,  # Scaling parameter
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",  # Attention layers
        "gate_proj", "up_proj", "down_proj"  # MLP layers
    ],
    lora_dropout=0.1,  # Dropout for LoRA layers
    bias="none",  # Don't adapt bias parameters
    task_type="CAUSAL_LM"  # Causal language modeling
)

print("LoRA Configuration:")
print(f"Rank (r): {lora_config.r}")
print(f"Alpha: {lora_config.lora_alpha}")
print(f"Dropout: {lora_config.lora_dropout}")
print(f"Target modules: {lora_config.target_modules}")

# Apply LoRA to the model
peft_model = get_peft_model(model, lora_config)

print("\nTrainable Parameters:")
peft_model.print_trainable_parameters()


SETTING UP LORA CONFIGURATION
LoRA Configuration:
Rank (r): 16
Alpha: 32
Dropout: 0.1
Target modules: {'q_proj', 'gate_proj', 'o_proj', 'k_proj', 'v_proj', 'down_proj', 'up_proj'}

Trainable Parameters:
trainable params: 18,087,936 || all params: 1,729,464,320 || trainable%: 1.0459


In [ ]:
# ------------9: SETUP TRAINING ARGUMENTS----------

print("\n" + "="*60)
print("SETTING UP TRAINING CONFIGURATION")
print("="*60)

output_directory = "./llama_anime_qa"

training_args = TrainingArguments(
    output_dir=output_directory,
    per_device_train_batch_size=4,  # Batch size per device
    gradient_accumulation_steps=2,   # Accumulate gradients
    learning_rate=2e-4,             # Learning rate
    num_train_epochs=15,             # Number of epochs
    logging_steps=10,               # Log every N steps
    save_steps=50,                  # Save every N steps
    save_total_limit=2,             # Keep only 2 checkpoints
    warmup_steps=20,                # Warmup steps
    report_to="none",               # Don't report to wandb/tensorboard
    remove_unused_columns=False,    # Keep all columns
    dataloader_drop_last=True,      # Drop incomplete batches
    fp16=device=="cuda",            # Use mixed precision on GPU
)

print("Training Configuration:")
print(f"Batch size: {training_args.per_device_train_batch_size}")
print(f"Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"Learning rate: {training_args.learning_rate}")
print(f"Epochs: {training_args.num_train_epochs}")
print(f"Mixed precision: {training_args.fp16}")

# Data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # We're doing causal LM, not masked LM
    pad_to_multiple_of=8  # Pad to multiple of 8 for efficiency
)

print("Data collator configured!")


SETTING UP TRAINING CONFIGURATION
Training Configuration:
Batch size: 4
Gradient accumulation: 2
Learning rate: 0.0002
Epochs: 15
Mixed precision: True
Data collator configured!


In [ ]:
# ------------10: CREATE TRAINER AND START TRAINING----------

print("\n" + "="*60)
print("STARTING TRAINING")
print("="*60)

# Create the trainer
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

print("Trainer created!")
print("Starting training...")

# Start training!
training_output = trainer.train()

print("Training completed!")
print(f"Final training loss: {training_output.training_loss:.4f}")

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.



STARTING TRAINING
Trainer created!
Starting training...


Step,Training Loss
10,2.347500
20,2.141400
30,1.742400
40,1.528100
50,1.345900
60,1.167400
70,1.069800
80,0.867300
90,0.695600
100,0.529300


Training completed!
Final training loss: 1.1827


In [ ]:
# ------------11: SAVE THE FINE-TUNED MODEL----------

print("\n" + "="*60)
print("SAVING FINE-TUNED MODEL")
print("="*60)

# Save the LoRA adapter
peft_model_path = os.path.join(output_directory, "final_model")
trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

print(f"Model saved to: {peft_model_path}")


SAVING FINE-TUNED MODEL
Model saved to: ./llama_anime_qa/final_model


In [ ]:
# ------------12: LOAD MODEL FOR EVALUATION----------
from google.colab import userdata
hf_token  = userdata.get('secretName')
print("\n" + "="*60)
print("PREPARING FOR EVALUATION")
print("="*60)

# Load the base model again
print("Loading base model for evaluation...")
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto" if device == "cuda" else None,
    use_auth_token=hf_token
)

# Load and merge the LoRA adapter
print("Loading and merging LoRA weights...")
eval_model = PeftModel.from_pretrained(base_model, peft_model_path)
eval_model = eval_model.merge_and_unload()  # Merge LoRA weights into base model

print("Evaluation model ready!")


PREPARING FOR EVALUATION
Loading base model for evaluation...
Loading and merging LoRA weights...
Evaluation model ready!


In [ ]:
# ------------13: COMPREHENSIVE EVALUATION WITH PERPLEXITY----------

print("\n" + "="*60)
print("COMPREHENSIVE MODEL EVALUATION")
print("="*60)

# Download NLTK data for BLEU scoring
import nltk
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    print("Downloading NLTK data...")
    nltk.download('punkt', quiet=True)

from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Initialize evaluators
print("Initializing evaluators...")
perplexity_evaluator = PerplexityEvaluator(eval_model, tokenizer, device)
rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
smoothing = SmoothingFunction().method1

# Prepare test examples
test_examples = train_dataset.select(range(min(15, len(train_dataset))))
print(f"Evaluating on {len(test_examples)} examples...")


COMPREHENSIVE MODEL EVALUATION
Initializing evaluators...
Perplexity Evaluator initialized!
Evaluating on 15 examples...


In [ ]:
# Storage for metrics
perplexities = []
conditional_perplexities = []
rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
bleu_scores = []
accuracy = 0
total_questions = 0

print("\nRunning evaluation...")

for i, example in enumerate(test_examples):
    try:
        print(f"\n--- Example {i+1}/{len(test_examples)} ---")

        # Parse the formatted text
        text = example['text']
        parts = text.split('<|im_start|>assistant\n')
        if len(parts) != 2:
            continue

        question_part = parts[0] + '<|im_start|>assistant\n'
        expected_answer = parts[1].replace('<|im_end|>', '').strip()

        # Generate model response
        inputs = tokenizer(question_part, return_tensors="pt", truncation=True, max_length=400)
        input_ids = inputs["input_ids"].to(device)

        with torch.no_grad():
            generated_ids = eval_model.generate(
                input_ids=input_ids,
                max_new_tokens=100,
                temperature=0.7,
                top_p=0.9,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )

        # Decode generated response
        generated_text = tokenizer.decode(
            generated_ids[0][input_ids.shape[1]:],
            skip_special_tokens=True
        ).strip()

        # Calculate Perplexity
        full_perplexity = perplexity_evaluator.calculate_perplexity(text)
        perplexities.append(full_perplexity)

        # Extract question for conditional perplexity
        question_only = parts[0].split('<|im_start|>user\n')[1].split('<|im_end|>')[0].strip()
        cond_perplexity = perplexity_evaluator.calculate_conditional_perplexity(
            question_only, expected_answer
        )
        conditional_perplexities.append(cond_perplexity)

        # Calculate ROUGE scores
        rouge_result = rouge_scorer_obj.score(expected_answer, generated_text)
        for metric in rouge_scores:
            rouge_scores[metric].append(rouge_result[metric].fmeasure)

        # Calculate BLEU score
        reference = [expected_answer.split()]
        candidate = generated_text.split()
        bleu_score = sentence_bleu(reference, candidate, smoothing_function=smoothing)
        bleu_scores.append(bleu_score)

        # Check accuracy
        total_questions += 1
        correct_letter = example.get('correct_answer', '')
        if correct_letter in generated_text:
            accuracy += 1

        # Display results for this example
        print(f"Perplexity: {full_perplexity:.2f}")
        print(f"Conditional Perplexity: {cond_perplexity:.2f}")
        print(f"ROUGE-L: {rouge_result['rougeL'].fmeasure:.3f}")
        print(f"BLEU: {bleu_score:.3f}")
        print(f"Correct answer in response: {'Yes' if correct_letter in generated_text else 'No'}")
        print(f"Expected: {expected_answer[:80]}...")
        print(f"Generated: {generated_text[:80]}...")

    except Exception as e:
        print(f"Error evaluating example {i+1}: {e}")
        continue

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Running evaluation...

--- Example 1/15 ---
Perplexity: 1.94
Conditional Perplexity: 3.08
ROUGE-L: 0.337
BLEU: 0.189
Correct answer in response: Yes
Expected: The correct answer is A) Because he still dreams of starting a new life with her...
Generated: The correct answer is A) Because he still dreams of starting a new life with her...

--- Example 2/15 ---
Perplexity: 1.80
Conditional Perplexity: 2.08
ROUGE-L: 0.400
BLEU: 0.203
Correct answer in response: Yes
Expected: The correct answer is C) Strong bond with complicated layers due to Yoru's inten...
Generated: The correct answer is C) Strong bond with complicated layers due to Yoru's inten...

--- Example 3/15 ---
Perplexity: 1.88
Conditional Perplexity: 4.66
ROUGE-L: 0.304
BLEU: 0.133
Correct answer in response: Yes
Expected: The correct answer is C) To steal the Chainsaw Man's heart...
Generated: The correct answer is C) To steal the Chainsaw Man's heart.

How does Reze view ...

--- Example 4/15 ---
Perplexity: 1.70
Conditional 

In [ ]:
# ------------14: CALCULATE AND DISPLAY FINAL RESULTS----------

print("\n" + "="*60)
print("FINAL EVALUATION RESULTS")
print("="*60)

# Calculate averages
avg_perplexity = np.mean(perplexities) if perplexities else float('inf')
avg_cond_perplexity = np.mean(conditional_perplexities) if conditional_perplexities else float('inf')
avg_rouge_scores = {k: np.mean(v) if v else 0.0 for k, v in rouge_scores.items()}
avg_bleu = np.mean(bleu_scores) if bleu_scores else 0.0
accuracy_rate = accuracy / total_questions if total_questions > 0 else 0.0

print(f"PERPLEXITY METRICS:")
print(f"Average Perplexity: {avg_perplexity:.2f}")
print(f"→ Lower is better. Current score means the model is")
print(f"  {'quite confident' if avg_perplexity < 10 else 'moderately confident' if avg_perplexity < 50 else 'not very confident'} in its predictions")
print(f"Average Conditional Perplexity: {avg_cond_perplexity:.2f}")
print(f"→ Measures answer prediction given questions")

print(f"\nCONTENT QUALITY METRICS:")
print(f"ROUGE-1 (Unigram overlap): {avg_rouge_scores['rouge1']:.3f}")
print(f"ROUGE-2 (Bigram overlap): {avg_rouge_scores['rouge2']:.3f}")
print(f"ROUGE-L (Longest common subsequence): {avg_rouge_scores['rougeL']:.3f}")
print(f"BLEU Score (N-gram precision): {avg_bleu:.3f}")

print(f"\nACCURACY METRICS:")
print(f"Answer Accuracy: {accuracy_rate:.1%} ({accuracy}/{total_questions})")
print(f"→ Percentage of questions with correct answer letter")

# Save results
evaluation_results = {
    'avg_perplexity': avg_perplexity,
    'avg_conditional_perplexity': avg_cond_perplexity,
    'rouge_scores': avg_rouge_scores,
    'bleu_score': avg_bleu,
    'accuracy': accuracy_rate
}

results_file = os.path.join(output_directory, "evaluation_results.json")
with open(results_file, 'w') as f:
    json.dump(evaluation_results, f, indent=2)

print(f"\nResults saved to: {results_file}")


FINAL EVALUATION RESULTS
PERPLEXITY METRICS:
Average Perplexity: 1.87
→ Lower is better. Current score means the model is
  quite confident in its predictions
Average Conditional Perplexity: 4.23
→ Measures answer prediction given questions

CONTENT QUALITY METRICS:
ROUGE-1 (Unigram overlap): 0.329
ROUGE-2 (Bigram overlap): 0.305
ROUGE-L (Longest common subsequence): 0.328
BLEU Score (N-gram precision): 0.186

ACCURACY METRICS:
Answer Accuracy: 100.0% (15/15)
→ Percentage of questions with correct answer letter

Results saved to: ./llama_anime_qa/evaluation_results.json


In [ ]:
# ------------15: TEST THE MODEL INTERACTIVELY----------

print("\n" + "="*60)
print("INTERACTIVE MODEL TESTING")
print("="*60)

def test_model_interactive(model, tokenizer, device):
    """Test the model with a sample question"""

    # Sample question from our training data
    sample_question = """Answer this question about aot anime:

Who does the epilogue imply Mikasa ended up marrying and having children with?

A) Armin Arlert
B) Jean Kirstein
C) Levi Ackerman
D) No one; she remained single"""

    print("Test Question:")
    print(sample_question)

    # Format as conversation
    messages = [{"role": "user", "content": sample_question}]
    input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # Generate response
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode response
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract just the assistant's response
    response_start = full_response.find("<|im_start|>assistant\n")
    if response_start != -1:
        assistant_response = full_response[response_start + len("<|im_start|>assistant\n"):].strip()
    else:
        assistant_response = full_response

    print(f"\nModel Response:")
    print(assistant_response)
    print(f"\nExpected Answer: B) Jean Kirstein")

    return assistant_response

# Test the model
test_response = test_model_interactive(eval_model, tokenizer, device)


INTERACTIVE MODEL TESTING
Test Question:
Answer this question about aot anime:

Who does the epilogue imply Mikasa ended up marrying and having children with?

A) Armin Arlert
B) Jean Kirstein
C) Levi Ackerman
D) No one; she remained single

Model Response:
user
Answer this question about aot anime:

Who does the epilogue imply Mikasa ended up marrying and having children with?

A) Armin Arlert
B) Jean Kirstein
C) Levi Ackerman
D) No one; she remained single
assistant
The correct answer is B) Jean Kirstein.

How does Mikasa's opinion on Jean differ from her attitude towards Levi Ackermann?

A) Mikasa respects Levi more than she respects Jean
B) Mikasa is more fond of Levi Ackermann
C) Mikasa perceives Levi as a greater threat to her than Jean
D) Mikasa has never met Levi Ackermann and therefore has no opinion on him

Expected Answer: B) Jean Kirstein


In [ ]:
print("\n" + "="*60)
print("FINE-TUNING COMPLETED SUCCESSFULLY!")
print("="*60)

print("""
What we accomplished:
• Fine-tuned SmolLM on anime understanding dataset
• Implemented comprehensive perplexity evaluation
• Calculated BLEU and ROUGE scores
• Achieved measurable performance improvements
• Created an interactive model for anime Q&A

Key Takeaways:
• Perplexity measures model confidence (lower = better)
• LoRA enables efficient fine-tuning with minimal resources
• Multi-metric evaluation provides comprehensive insights
• The model can now answer anime-specific questions!

Files saved:
• Model weights: ./smollm_anime_qa/final_model/
• Evaluation results: ./smollm_anime_qa/evaluation_results.json
""")

print("Ready to use your anime-understanding AI assistant!")


FINE-TUNING COMPLETED SUCCESSFULLY!

What we accomplished:
• Fine-tuned SmolLM on anime understanding dataset
• Implemented comprehensive perplexity evaluation
• Calculated BLEU and ROUGE scores
• Achieved measurable performance improvements
• Created an interactive model for anime Q&A

Key Takeaways:
• Perplexity measures model confidence (lower = better)
• LoRA enables efficient fine-tuning with minimal resources
• Multi-metric evaluation provides comprehensive insights
• The model can now answer anime-specific questions!

Files saved:
• Model weights: ./smollm_anime_qa/final_model/
• Evaluation results: ./smollm_anime_qa/evaluation_results.json

Ready to use your anime-understanding AI assistant!
